# Vive La Campagne! 

Ce notebook a pour objectif de selectionner et visualiser les communes 

## Communes

In [67]:
from urllib.request import urlopen
import json
import pandas as pd
from datetime import datetime
from random import uniform
from time import sleep

Pour ce faire nous commençons par importer une basée de données reprennant toutes les communes de France:
[Base de données villes française CC-BY-SA](https://sql.sh/736-base-donnees-villes-francaises)


In [68]:
communes = pd.read_csv("data/villes_france.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
communes.head(3)

,ville_id,ville_departement,ville_slug,ville_nom,ville_nom_simple,ville_nom_reel,ville_nom_soundex,ville_nom_metaphone,ville_code_postal,ville_commune,...,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_longitude_grd,ville_latitude_grd,ville_longitude_dms,ville_latitude_dms,ville_zmin,ville_zmax
0,1,01,ozan,OZAN,ozan,Ozan,O250,OSN,01190,284,...,93,6.6,4.91667,46.3833,2866,51546,+45456,462330,170,205
1,2,01,cormoranche-sur-saone,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,C65652625,KRMRNXSRSN,01290,123,...,107,9.85,4.83333,46.2333,2772,51379,+44953,461427,168,211
2,3,01,plagne-01,PLAGNE,plagne,Plagne,P425,PLKN,01130,298,...,20,6.2,5.73333,46.1833,3769,51324,+54342,461131,560,922


Sur les 27 colomnes, nous garderons que les suivantes:

In [69]:
colmnuns = [
    "ville_id",
    "ville_departement",
    "ville_nom",             # En majuscule sans accents
    "ville_nom_simple",      # En minuscule sans accents ni tirets
    "ville_nom_reel", 
    "ville_code_postal",
    "ville_commune",
    "ville_code_commune",    # ou INSEE
    "ville_arrondissement",
    "ville_canton",
    "ville_amdi",
    "ville_population_2010",
    "ville_population_1999",
    "ville_population_2012",
    "ville_densite_2010",
    "ville_surface",
    "ville_longitude_deg",
    "ville_latitude_deg",
    "ville_zmax"              # L'altitude
]           

communes = communes.filter(colmnuns)
communes.head(3)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,ville_amdi,ville_population_2010,ville_population_1999,ville_population_2012,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_zmax
0,1,01,OZAN,ozan,Ozan,01190,284,01284,2,26,6,618,469,500,93,6.6,4.91667,46.3833,205
1,2,01,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,01290,123,01123,2,27,6,1058,903,1000,107,9.85,4.83333,46.2333,211
2,3,01,PLAGNE,plagne,Plagne,01130,298,01298,4,03,6,129,83,100,20,6.2,5.73333,46.1833,922


Maintenant nous allons reprendre uniquement les communes des départements suivants:

In [70]:
departements = [
    "31",  # Haute-Garonne
    "32",  # Gers
    "65",  # Hautes-Pyrénées
    "81",  # Tarn
]

toulouse_counrtyside = communes.loc[communes['ville_departement'].isin(departements)]
toulouse_counrtyside.head(3)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,ville_amdi,ville_population_2010,ville_population_1999,ville_population_2012,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_zmax
11514,11515,31,LUNAX,lunax,Lunax,31350,307,31307,2,06,6,60,56,100,11,5.1,0.683333,43.3333,298
11515,11516,31,FONTENILLES,fontenilles,Fontenilles,31470,188,31188,1,30,6,4505,2920,3900,222,20.22,1.2,43.55,280
11516,11517,31,ARBON,arbon,Arbon,31160,012,31012,2,01,6,92,76,100,20,4.47,0.75,43,896


In [71]:
toulouse_counrtyside.count()

ville_id                 1849
ville_departement        1849
ville_nom                1849
ville_nom_simple         1849
ville_nom_reel           1849
ville_code_postal        1849
ville_commune            1849
ville_code_commune       1849
ville_arrondissement     1849
ville_canton             1849
ville_amdi               1849
ville_population_2010    1849
ville_population_1999    1849
ville_population_2012    1849
ville_densite_2010       1849
ville_surface            1849
ville_longitude_deg      1849
ville_latitude_deg       1849
ville_zmax               1849
dtype: int64

In [72]:
gares = pd.read_csv("data/localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv", sep=';', error_bad_lines=False, index_col=False, dtype='unicode')
departements = [
    "31",  # Haute-Garonne
    "32",  # Gers
    "65",  # Hautes-Pyrénées
    "81",  # Tarn
        ]

codeuic = [
"87615476",
"87615484",
"87615468",
"87615526",
"87775197",
"87328021",
"87615542",
"87615013",
"87618215",
"87611723",
"87618207",
"87611707",
"87612028",
"87612002",
"87616011",
"87611400",
"87611301",
"87611665",
"87611657",
"87446179",
"87353573",
"87611467",
"87611962",
"87611004",
"87618116",
"87611384",
"87353581",
"87611434",
"87611954",
"87611392",
"87353599",
"87611947",
"87497461",
"87612010",
"87611939",
"87611376",
"87611921"
]

gares = gares.loc[gares['departement'].isin(departements)]
#gares = gares.loc[~gares['codeuic'].isin(codeuic)]

In [52]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in gares['geo_point_2d']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
gares['latitude'] = lat
gares['longitude'] = lon

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,latitude,longitude
11,46,2064,Gare de voyageurs,L'Isle-Jourdain,gare avec accueil du public,87611806,Occitanie,32,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.0897045076...","43.6173114196,1.08970450764",43.6173114196,1.08970450764
13,51,2088,Gare de voyageurs et de fret,Cazères-sur-Garonne,gare avec accueil du public,87611079,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.0823282310...","43.2127599987,1.08232823102",43.2127599987,1.08232823102
18,68,2255,Gare de voyageurs,Aubiet,halte ferroviaire,87611764,Occitanie,32,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.7892317734...","43.6500715921,0.789231773446",43.6500715921,0.789231773446
38,177,3366,Gare de voyageurs et de fret,Lexos,halte ferroviaire,87613489,Occitanie,81,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.8850624224...","44.1422304256,1.88506242248",44.1422304256,1.88506242248
41,194,3655,Gare de voyageurs,Marignac-Saint-Béat,halte ferroviaire,87611194,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.6521641313...","42.9157573682,0.652164131336",42.9157573682,0.652164131336
50,245,4233,Gare de voyageurs,Saléchan-Siradan,halte ferroviaire,87618710,Occitanie,65,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.6316603800...","42.9630705836,0.63166038007",42.9630705836,0.63166038007
51,246,4238,Gare de voyageurs,Mérenvielle,halte ferroviaire,87611814,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.1515214744...","43.6368842109,1.15152147448",43.6368842109,1.15152147448
59,291,4779,Gare de voyageurs,Gimont-Cahuzac,gare avec accueil du public,87611772,Occitanie,32,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.8676288726...","43.6331940821,0.86762887265",43.6331940821,0.86762887265
69,60,2222,Gare de voyageurs,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",42.7971107969,0.596599645992
74,95,2560,Gare de voyageurs et de fret,Montréjeau-Gourdan-Polignan,gare avec accueil du public,87611152,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5735091098...","43.0780437575,0.573509109825",43.0780437575,0.573509109825


Let's see the distance between two points

In [73]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [float(lon1), float(lat1), float(lon2), float(lat2)])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


In [74]:
df_all = pd.merge(toulouse_counrtyside.assign(key=0), gares.assign(key=0), on='key').drop('key', axis=1)

In [76]:
df_all['DISTANCE_GARE'] = df_all.apply(
    (lambda row: haversine(
        row['ville_latitude_deg'], row['ville_longitude_deg'],
        row['latitude'], row['latitude']
    )),
    axis=1
)

KeyError: 'latitude'

In [56]:
closest = df_all.loc[df_all.groupby(["ville_id", 'ville_latitude_deg', 'ville_longitude_deg'])["DISTANCE_GARE"].idxmin()]

In [57]:
toulouse_counrtyside_w_dist = toulouse_counrtyside.merge(
    closest,
    on=["ville_id", 'ville_latitude_deg', 'ville_longitude_deg'],
    suffixes=('', '_cl')
).drop(['latitude', 'latitude'], axis=1)

In [58]:
toulouse_counrtyside_w_dist.sort_values(by=['DISTANCE_GARE']).head(7)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,...,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,longitude,DISTANCE_GARE
1656,33177,81,MURAT-SUR-VEBRE,murat sur vebre,Murat-sur-Vèbre,81320,192,81192,2,22,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4441.859649
1711,33232,81,BARRE,barre,Barre,81320,023,81023,2,22,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4445.465577
1677,33198,81,MOULIN-MAGE,moulin mage,Moulin-Mage,81320,188,81188,2,22,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4447.884542
1549,33070,81,NAGES,nages,Nages,81320,193,81193,2,22,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4450.590770
1680,33201,81,LACAUNE,lacaune,Lacaune,81230,124,81124,2,15,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4459.528986
1564,33085,81,SAINT-JUERY,saint juery,Saint-Juéry,81160,257,81257,1,36,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4459.979213
1655,33176,81,LABASTIDE-ROUAIROUX,labastide rouairoux,Labastide-Rouairoux,81270,115,81115,2,28,...,Luchon,gare avec accueil du public,87611236,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [0.5965996459...","42.7971107969,0.596599645992",0.596599645992,4464.649605


In [25]:
toulouse_counrtyside_w_dist.to_csv('data/tls-communes-gares.csv', encoding='utf-8')

Yes on a maintenant nos communes qui nous intéresse!

In [5]:
durations = {
    'ville_code_commune': [],
    'duration_toulouse': [], 
    'distance_toulouse': [], 
    'base_duration_toulouse': []
}
toulouse_coord= '43.6,1.43333'
api_key = 'uJLrVPP4drsWQQjj3nqWDbG1W-FKZfsmlvg_SZxGyWs'

for i,j in toulouse_counrtyside.iterrows():
    lat = j['ville_latitude_deg']
    long = j['ville_longitude_deg']

    url = 'https://router.hereapi.com/v8/routes?transportMode=car&origin='+str(lat)+','+str(long)+'&destination='+toulouse_coord+'&return=summary&apiKey='+api_key

    res = urlopen(url).read()
    res_json = json.loads(res)

    sum = (res_json.get('routes')[0]['sections'][0]['summary'])
    
    durations['ville_code_commune'].append(j['ville_code_commune'])
    durations['duration_toulouse'].append(sum['duration'])
    durations['distance_toulouse'].append(sum['length'])
    durations['base_duration_toulouse'].append(sum['baseDuration'])
    
    sleep(uniform(0.5,0.1))

    
durations = pd.DataFrame(durations)
durations.to_csv('data/durations.csv', encoding='utf-8')
durations.head(10)

KeyboardInterrupt: 

### A vous : 

   - Quelle a été la plus grande subvention allouée en 2011 ? 
   - Quel est le parlementaire qui a reçu le plus grand nombre de subventions ? 
   - Quel est le parlementaire qui a reçu le plus de subventions en valeur ? 
   - Quel est le département qui a reçu le plus de subventions ? 

In [ ]:
reserve = pd.read_csv("data/reserve.csv")
reserve.head(2)

Nous pouvons appliquer plusieurs clés : **quel est le nombre de subventions par partis politiques et par chambre ?**

In [ ]:
chambres = reserve['Subvention allouée'].groupby([reserve['Groupe politique du parlementaire'],reserve['Nature']]).count()
chambres

Nous créons dans ce cas une série avec deux index (partis et nature). 

Nous pouvons faire 'déplier' cette série avec la méthode **unstack** :

In [ ]:
groupes_chambres = chambres.unstack()
groupes_chambres

Nous pouvons également grouper sans spécifier de colonne. Si les données à grouper se trouvent dans mon DataFrame, je peux grouper de la façon suivante : 

In [ ]:
reserve.groupby('Nature').count()

(Attention aux aggrégations hâtives ;))

### A vous : 

- Quelle est la moyenne des coûts de projets par type d'assemblée ? 
- Combien de projets le Sénat a - t-il financé dans les Yvelines ? 
- Quelle a été la subvention allouée pour à Paris ? 

Pandas nous permet également de passer des dictionnaires ou des séries au sein des groupes. 

Cela est intéressant lorsque nous souhaitons faire un mapping entre les colonnes (ou les lignes). 

Dans notre cas, les acronymes entre l'Assemblée Nationale et le Sénat varient (i.e : SOC et SRC désignent tous deux les groupes du Parti Socialiste). On peut donc écrire la correspondance suivante : 

In [ ]:
mapping = {'CRC':'Partis de Gauche', 
           'CRC-SPG':'Partis de Gauche', 
           'ECO':'Ecologistes', 
           'GDR':'Radicaux', 
           'NC':'Centristes', 
           'NI':'Non Inscrits', 
           'RDSE':'Radicaux',
           'SOC':'Parti Socialiste',
           'SOCV':'Parti Socialiste',
           'SRC':'Parti Socialiste',
           'UC':'Centristes', 
           'UDI':'Centristes', 
           'UMP':'Union Mouvement Populaire'}

Dans ce cas, nous souhaitons grouper selon la correspondance des index (axis =0) du DataFrame **groupes_chambres** avec le dictionnaire **mapping**, soit : 

In [ ]:
groupes_chambres

In [ ]:
groupes_chambres.groupby(mapping, axis=0).sum()

Une autre façon de réaliser facilement des mapping, est d'utiliser la méthode **map** sur une série. Dans notre cas : 

In [ ]:
reserve['parti'] = reserve['Groupe politique du parlementaire'].map(mapping)
reserve.head(2)

## Agrégation

L'aggrégation de données correspond à une transformation d'un tableau de données en une valeur. 

Pandas permet d'accéder à des agrégations génériques (moyennes, somme etc..), mais il est également possible de créer ses propres fonctions d'agrégation. 

Il est ainsi possible d'aggréger ses données en utilisant plusieurs fonctions d'agrégations. Déclarons par exemple les trois fonctions suivantes moyenne, maximum et minimum : 

In [ ]:
functions = ['mean','max','min']

Nous pouvons appeler la méthode **agg** qui appliquera pour chacun des groupes les fonctions appelées. 

Si nous souhaitons par exemple afficher la moyenne, le maximum et le minimum des subventions allouée par parti politique, nous pouvons écrire : 

In [ ]:
reserve.groupby('parti').agg(functions)

Il est également possible de définir ses propres aggrégations grâce à la méthode **apply**

Comme précisé précédemment, la méthode **apply** sépare un set de données suivant les clés spécifier dans la méthode **groupby** puis applique la fonction appelée en paramètre. 

Nous défninissons la fonction **part** qui renvoie la moyenne du ratio subvention allouée / coût du projet : 

In [ ]:
def part(df):
    return np.mean(df['Subvention allouée']/df['Coût du projet'])

La moyenne totale de la part des projets subventionnées peut donc s'écrire : 

In [ ]:
part(reserve)

Il est dès lors possible d'appliqer la fonction **part** avec un groupby pour obtenir la part des subventions par Département : 

In [ ]:
reserve.groupby('Département').apply(part).sort_values(ascending=False)

### A vous : 

- Quel est la part subventionnée des projets par partis politiques ? 
- Quel parlementaire, ayant réalisé plus de 50 subventions, a réalisé le plus de subventions dans sa région ? 
- Quel est le parti politique qui a réalisé le plus de subventions dans sa région ? 